# Week 4 Automate Parametric Simulations

In [ ]:
import pandas as pd
import numpy as np
from eppy.modeleditor import IDF
from eppy import modeleditor
import esoreader
import os
import sys
from itertools import product
import shutil

In [ ]:
eplus_root = r"C:\EnergyPlusV25-1-0" # Change this to your EnergyPlus root directory
iddfile = os.path.join(eplus_root,"Energy+.idd")
try:
    IDF.setiddname(iddfile)
except modeleditor.IDDAlreadySetError as e:
    print(e)
    
root_dir = os.getcwd()

MAP_WEATHER = {
    "San Francisco":os.path.normpath(os.path.join(root_dir, r'.\weather data\USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw')),
    "Sacramento":os.path.normpath(os.path.join(root_dir, r"..\weather data\USA_CA_Sacramento.Exec.AP.724830_TMY3.epw")),
    "Chicago":os.path.normpath(os.path.join(root_dir, r"..\weather data\USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw")),
    "New York":os.path.normpath(os.path.join(root_dir, r"..\weather data\USA_NY_New.York-J.F.Kennedy.Intl.AP.744860_TMY3.epw")),
}

In [ ]:
class ESO:
    def __init__(self, path):
        self.dd, self.data = esoreader.read(path)
    def read_var(self, variable, frequency = "Hourly"):
        return [
            {"key": k,
             "series": self.data[self.dd.index[frequency, k, variable]]}
            for _f, k, _v in self.dd.find_variable(variable)
        ]
    def get_df(self, variable, frequency = "Hourly"):
        dic = self.read_var(variable, frequency)
        key = [each["key"] for each in dic]
        values = [each["series"] for each in dic]
        df = pd.DataFrame(values,index = key).T
        return df
    def total_kwh(self, variable, frequency = "Hourly"):
        j_per_kwh = 3_600_000
        results = self.read_var(variable,frequency)
        return sum(sum(s["series"]) for s in results)/j_per_kwh

In [ ]:
# Step 1: Read the parametric table
param_table = pd.read_csv()

# Step 2: Prepare result DataFrame
result_df = param_table.copy().assign(Heating_kwh=None, Cooling_kwh=None, Lighting_kwh=None)

# Step 3: Run simulations looping through each row of the parametric table
for index, row in param_table.iterrows():
    
    sim_id = row[...]
    wea = row[...]
    mod = row[...]
    
    # Locate IDF and EPW files
    idf_dir = os.path.join()
    epw_dir = MAP_WEATHER[...]
    
    # Load the IDF model
    idf = IDF(...)
    
    # Add a meter for lighting electricity
    idf.newidfobject(... )
    
    # Create simulation directory    
    sim_dir = os.path.join(...)
    os.makedirs(sim_dir, exist_ok=True)
    
    # Save and run new idf 
    idf.saveas(...)
    os.chdir(...)
    
    try:
        idf.run(expandobjects=True)
        
         # Read the eso file
        eso_dir = os.path.join(...)
        eso = ESO(eso_dir)
        
        # Update variable names if your meters differ
        result_df.loc[index, "Heating_kwh"] = eso.total_kwh("DistrictHeatingWater:Facility","TimeStep")
        result_df.loc[index, "Cooling_kwh"] = eso.total_kwh(...)
        result_df.loc[index, "Lighting_kwh"] = eso.total_kwh(...)
            
        print(f'sim {sim_id} done')
    
    except Exception as e:
        print(f"⚠️ Simulation {sim_id} failed: {e}")
    
    os.chdir(root_dir)

# Step 4: Save the result dataframe
result_df.to_csv(... , index= ...)  